# 1.0 Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

## 1.1 Funções

In [2]:
# Função para calcular métricas
def calcular_metricas(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average='weighted'),
        "Recall": recall_score(y_true, y_pred, average='weighted'),
        "F1-Score": f1_score(y_true, y_pred, average='weighted')
    }

# Função para realizar Grid Search
def grid_search(model, param_grid, X_train, y_train):
    grid_search = GridSearchCV(model, param_grid)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

# 2.0 Leitura dos Dados

In [3]:
x_train = pd.read_csv( '../datasets/classification/X_training.csv' )
y_train = pd.read_csv( '../datasets/classification/y_training.csv' )

x_val = pd.read_csv( '../datasets/classification/X_validation.csv' )
y_val = pd.read_csv( '../datasets/classification/y_validation.csv' )

x_test = pd.read_csv( '../datasets/classification/X_test.csv' )
y_test = pd.read_csv( '../datasets/classification/y_test.csv' )

## 2.1 Verificando melhores parametros para treinamento do modelo.

In [4]:
# Definindo os modelos
rf_model = RandomForestClassifier()
knn_model = KNeighborsClassifier()
logreg_model = LogisticRegression()
dt_model = DecisionTreeClassifier()

# Definindo as grades de parâmetros
rf_param_grid = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30]}
knn_param_grid = {'n_neighbors': [3, 5, 7, 9]}
logreg_param_grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'max_iter': [100, 200, 300]}
dt_param_grid = {'max_depth': [None, 10, 20, 30, 40]}

In [5]:
# Random Forest
rf_best_params = grid_search(rf_model, rf_param_grid, x_train, y_train)
print("Melhores parâmetros para Random Forest:", rf_best_params)

# K-Neighbors
knn_best_params = grid_search(knn_model, knn_param_grid, x_train, y_train)
print("Melhores parâmetros para KNN:", knn_best_params)

# Logistic Regression
logreg_best_params = grid_search(logreg_model, logreg_param_grid, x_train, y_train)
print("Melhores parâmetros para Logistic Regression:", logreg_best_params)

# Decision Tree
dt_best_params = grid_search(dt_model, dt_param_grid, x_train, y_train)
print("Melhores parâmetros para Decision Tree:", dt_best_params)

Melhores parâmetros para Random Forest: {'max_depth': 30, 'n_estimators': 200}
Melhores parâmetros para KNN: {'n_neighbors': 3}
Melhores parâmetros para Logistic Regression: {'max_iter': 100, 'solver': 'newton-cg'}
Melhores parâmetros para Decision Tree: {'max_depth': 10}


# 3.0 Algoritmos de Machine Learning

In [6]:
# Definindo os modelos
modelos = {
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Decision Tree": DecisionTreeClassifier(max_depth=10),
    "Random Forest": RandomForestClassifier(max_depth=30, n_estimators=200),
    "Logistic Regression": LogisticRegression(max_iter=100, solver='newton-cg')
}
# Treinando os modelos e armazenando as métricas de treinamento e validação
resultados_treinamento = []
resultados_validacao = []

for nome, modelo in modelos.items():
    # Treinamento
    modelo.fit(x_train, y_train)
    y_pred_treino = modelo.predict(x_train)
    metricas_treino = calcular_metricas(y_train, y_pred_treino)
    resultados_treinamento.append({"Modelo": nome, **metricas_treino})
    print(f"Modelo {nome} treinado.")

    # Validação
    y_pred_val = modelo.predict(x_val)
    metricas_val = calcular_metricas(y_val, y_pred_val)
    resultados_validacao.append({"Modelo": nome, **metricas_val})

Modelo KNN treinado.
Modelo Decision Tree treinado.
Modelo Random Forest treinado.
Modelo Logistic Regression treinado.


# 4.0 Performance

In [7]:
# Criando tabelas de resultados
tabela_treinamento = pd.DataFrame(resultados_treinamento)
tabela_validacao = pd.DataFrame(resultados_validacao)

# Exibindo os resultados
print("Resultados de Treinamento:")
print(tabela_treinamento)
print("\nResultados de Validação:")
print(tabela_validacao)

Resultados de Treinamento:
                Modelo  Accuracy  Precision    Recall  F1-Score
0                  KNN  0.832186   0.831913  0.832186  0.831998
1        Decision Tree  0.955444   0.955642  0.955444  0.955349
2        Random Forest  1.000000   1.000000  1.000000  1.000000
3  Logistic Regression  0.876081   0.875951  0.876081  0.875757

Resultados de Validação:
                Modelo  Accuracy  Precision    Recall  F1-Score
0                  KNN  0.676277   0.675892  0.676277  0.676070
1        Decision Tree  0.947006   0.947177  0.947006  0.946891
2        Random Forest  0.964864   0.965059  0.964864  0.964796
3  Logistic Regression  0.874385   0.874241  0.874385  0.874061


# Dados de Teste

In [8]:
# Avaliação no conjunto de teste
resultados_teste = []

for nome, modelo in modelos.items():
    y_pred_teste = modelo.predict(x_test)
    metricas_teste = calcular_metricas(y_test, y_pred_teste)
    resultados_teste.append({"Modelo": nome, **metricas_teste})

tabela_teste = pd.DataFrame(resultados_teste)
print("\nResultados no Conjunto de Teste:")
print(tabela_teste)


Resultados no Conjunto de Teste:
                Modelo  Accuracy  Precision    Recall  F1-Score
0                  KNN  0.672228   0.671287  0.672228  0.671644
1        Decision Tree  0.945931   0.946050  0.945931  0.945838
2        Random Forest  0.964006   0.964173  0.964006  0.963946
3  Logistic Regression  0.871510   0.871400  0.871510  0.871202
